In [1]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [2]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
# from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
# python_repl_tool = PythonREPLTool()


In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def create_agent(
        llm: ChatOpenAI, tools: list, system_prompt: str
):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


In [4]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [11]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Positive Pointer", "Negative Pointer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
options = ["FINISH"] + members
function_def ={
    "name": "route",
    "description": "Select the next role.",
    "parameters":{
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview", max_tokens=1024)

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [12]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

positive_agent = create_agent(llm, [tavily_tool], "You should provide accurate information about what points of the image are causing good CTR.")
positive_node = functools.partial(agent_node, agent=positive_agent, name="Positive Pointer")

negative_agent = create_agent(llm, [tavily_tool], "You should provide accurate information about what points of the image are causing bad CTR.")
negative_node = functools.partial(agent_node, agent=negative_agent, name="Negative Pointer")

workflow = StateGraph(AgentState)
workflow.add_node("Positive Pointer", positive_node)
workflow.add_node("Negative Pointer", positive_node)
workflow.add_node("supervisor", supervisor_chain)

In [13]:
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [14]:
import base64

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

# Example usage
image_path = "../thumbnail.jpg"
base64_image = encode_image_to_base64(image_path)


In [16]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                content=[
                    {
                        "type": "text",
                        "text": "Describe the image below,"
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ]
            )
        ],
    },
    {"recursion_limit":150}
)

BadRequestError: Error code: 400 - {'error': {'message': '2 validation errors for Request\nbody -> function_call\n  extra fields not permitted (type=value_error.extra)\nbody -> functions\n  extra fields not permitted (type=value_error.extra)', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [15]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content=[
                    {
                        "type": "text",
                        "text": "Describe the image below,"
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ]
            )
        ],
    },
    {"recursion_limit":150}
):
    print(s)
    print("------")

BadRequestError: Error code: 400 - {'error': {'message': '2 validation errors for Request\nbody -> function_call\n  extra fields not permitted (type=value_error.extra)\nbody -> functions\n  extra fields not permitted (type=value_error.extra)', 'type': 'invalid_request_error', 'param': None, 'code': None}}